# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [92]:
import pandas as pd

# Чтение данных из Excel файла
df = pd.read_excel('себестоимостьА_в1.xlsx', sheet_name='Рецептура', index_col=0, skiprows=2)

# Рассчет себестоимости для каждого вида продукции
df['Себестоимость'] = (
    df['Вода'] * df['Цена ресурсов, руб.'][0] * df['Вес, кг.'] +
    df['Пшеничная мука'] * df['Цена ресурсов, руб.'][1] * df['Вес, кг.'] +
    df['Дрожжи'] * df['Цена ресурсов, руб.'][2] * df['Вес, кг.'] +
    df['Сахар'] * df['Цена ресурсов, руб.'][3] * df['Вес, кг.'] +
    df['Соль'] * df['Цена ресурсов, руб.'][4] * df['Вес, кг.'] +
    df['Куркума молотая'] * df['Цена ресурсов, руб.'][5] * df['Вес, кг.'] +
    df['Топленное масло'] * df['Цена ресурсов, руб.'][6] * df['Вес, кг.'] +
    df['Кунжутные семена'] * df['Цена ресурсов, руб.'][7] * df['Вес, кг.'] +
    df['Масло оливковое'] * df['Цена ресурсов, руб.'][8] * df['Вес, кг.']
)

# Вывод рассчитанных данных
print(df[['Наименование продукции', 'Себестоимость']])


KeyError: 'Вода'

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [21]:
import pandas as pd

reviews_df = pd.read_csv('reviews_sample.csv', index_col=0)


recipes_df = pd.read_csv('recipes_sample.csv', index_col='id')


recipes_df = recipes_df[['name', 'minutes', 'submitted', 'description', 'n_ingredients']]


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [22]:
recipes_df = pd.read_csv('recipes_sample.csv', index_col='id')
reviews_df = pd.read_csv('reviews_sample.csv', index_col=0)

# выбираем случайные 5% строк из каждой таблицы
recipes_sample = recipes_df.sample(frac=0.05)
reviews_sample = reviews_df.sample(frac=0.05)

with pd.ExcelWriter('recipes.xlsx') as writer:
    recipes_sample.to_excel(writer, sheet_name='Рецепты')
    reviews_sample.to_excel(writer, sheet_name='Отзывы')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [23]:
df = pd.read_excel('recipes.xlsx', sheet_name='Рецепты')

# добавляем столбец со временем выполнения рецепта в секундах
df['seconds_assign'] = df['minutes'] * 60

wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# устанавливаем заголовок для столбца
sheet.range('I1').value = 'seconds_assign'

# присваиваем массив значений диапазону ячеек столбца
sheet.range('I2').value = df['seconds_assign'].values.reshape(-1, 1)

wb.save()
wb.close()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [30]:
import xlwings as xw


wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

sheet.range('J1').value = 'seconds_formula'
sheet.range('J2').formula = '=C2*60'

last_row = sheet.range('B' + str(sheet.cells.last_cell.row)).end('up').row
sheet.range('J2:J' + str(last_row)).formula = '=C2*60'

wb.save()
wb.close()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [24]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# получаем диапазон для заголовков столбцов
header_range = sheet.range('I1:J1')

# устанавливаем стиль для заголовков столбцов
header_range.api.Font.Bold = True
header_range.api.HorizontalAlignment = 2

wb.save()
wb.close()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [25]:
wb = xw.Book('recipes.xlsx')
ws = wb.sheets['Рецепты']

# Задаем диапазон столбца minutes
minutes_range = ws.range('C2:C1501')

# Цикл для раскраски ячеек в соответствии с условиями
for cell in minutes_range:
    if cell.value < 5:
        cell.color = (146, 208, 80)  # зеленый
    elif 5 <= cell.value < 10:
        cell.color = (255, 255, 0)  # желтый
    else:
        cell.color = (255, 0, 0)  # красный

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [40]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
# Добавляем столбец n_reviews
sheet.range('K1').value = 'n_reviews'
sheet.range('K2:K{}'.format(sheet.range('A1').current_region.last_cell.row)).formula = '=COUNTIFS(Отзывы!A:A, Рецепты!A2)'

# устанавливаем стиль для заголовков столбцов
header_range = sheet.range('K1')
header_range.api.Font.Bold = True
header_range.api.HorizontalAlignment = 2

wb.save()
wb.close()



## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
import xlwings as xw

def validate():
    
    wb = xw.Book('recipes.xlsx')
    

    recipes_sheet = wb.sheets['Рецепты']
    reviews_sheet = wb.sheets['Отзывы']
    

    last_recipe_row = recipes_sheet.range('A' + str(recipes_sheet.cells.last_cell.row)).end('up').row
    last_review_row = reviews_sheet.range('A' + str(reviews_sheet.cells.last_cell.row)).end('up').row
    
    
    reviews_sheet.range('A2:E' + str(last_review_row)).color_index = None
    

    for i in range(2, last_review_row + 1):
        rating = reviews_sheet.range('C' + str(i)).value
        recipe_id = reviews_sheet.range('B' + str(i)).value
        
    
        if rating is None or not (0 <= rating <= 5):
            reviews_sheet.range('E' + str(i)).color_index = 3
        
    
        recipe_found = False
        for j in range(2, last_recipe_row + 1):
            if recipes_sheet.range('A' + str(j)).value == recipe_id:
                recipe_found = True
                break
        if not recipe_found:
            reviews_sheet.range('B' + str(i)).color_index = 3
            reviews_sheet.range('C' + str(i)).color_index = 3
            
   
    wb.save()
    wb.close()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [65]:
import xlwings as xw
import csv

filename = 'recipes_model.xlsx'

if not xw.Book(filename).sheets:
    # создание новой книги
    wb = xw.Book()
    wb.sheets.add(name='Модель')
    wb.save(filename)
    wb.close()

with open('recipes_model.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    data = list(reader)

wb = xw.Book(filename)
sheet = wb.sheets['Модель']

sheet.clear()


sheet.range('A1').value = ['Название столбца', 'Тип данных', 'Обязательно к заполнению', 'Ключ', 'Описание']


for row_idx, row in enumerate(data):
    sheet.range(f'A{row_idx+2}').value = row

sheet.range('A1:E1').color = (0, 128, 128)
sheet.range('A1:E1').api.Font.Bold = True
sheet.autofit()

wb.save()
wb.close()


10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
import xlwings as xw

wb = xw.Book('recipes_model.xlsx')
ws = wb.sheets['Модель']
num_rows = ws.range('B:B').end('down').row - 1
formula = f'=B2 & " " & UPPER(C2) & IF(D2="PK", " PRIMARY KEY", IF(D2="FK", " REFERENCES " & H2 & "(" & I2 & ")", IF(D2<>"PK" & F2="Y", " NOT NULL", "")))'
ws.range('J2').value = formula
ws.range(f'J2:J{num_rows+1}').api.AutoFill(ws.range(f'J2:J{num_rows+1}').api, 0)
wb.save()
wb.close()


11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
import xlwings as xw


wb = xw.Book("recipes_model.xlsx")
sheet = wb.sheets["Модель"]


header_style = xw.Style(
    fill=xw.PatternFill(pattern="Solid", fgColor="00ccff"),
    font=xw.Font(bold=True)
)


header_range = sheet.range("A1:J1")
header_range.style = header_style
header_range.autofit()


table_range = sheet.range("A1:J20")  
table_range.api.AutoFilter()  


wb.save()
wb.close()


12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 